###### Name: Deepak Vadithala
###### Course: MSc Data Science
###### Project Name: MOOC Recommender System

##### Notes:
This notebook contains the analysis of the **Cosine Similarity** model. 
Single variable (Skills Score) is used to predict the course category.
Each skill has a weighting within the respective role and the weighting is applied after calculating the **Cosine Similarity** score.Skill Score is calculated using the similarity between the skills from LinkedIn compared with the course description from Coursera.


*Model Source Code Path: /mooc-recommender/Model/Cosine_Distance.py*

*Github Repo: https://github.com/iamdv/mooc-recommender*

In [1]:
# **************************** IMPORTANT ****************************
'''
This cell configuration settings for the Notebook. 
You can run one role at a time to evaluate the performance of the model
Change the variable names to run for multiple roles

In this model - cosine distance is calculated between the skills and
the course description. Each skill has a weighted score based on the 
popularity of the skill. This is derived by endorsements of the respective
skill by other linkedin connections.
'''


# *******************************************************************
# For each role a list of category names are grouped. 
# Please don't change these variables

label_DataScientist = ['Data Science','Data Analysis','Data Mining','Data Visualization']

label_SoftwareDevelopment = ['Software Development','Computer Science',
                           'Programming Languages', 'Algorithms and Data Structures', 
                           'Information Technology']


label_DatabaseAdministrator = ['Databases']

label_Cybersecurity = ['Cybersecurity']

label_FinancialAccountant = ['Finance', 'Accounting']

label_MachineLearning = ['Machine Learning', 'Deep Learning']

label_Musician = ['Music']

label_Dietitian = ['Nutrition & Wellness']

label_Psychologist = ['Psychology']
            
# *******************************************************************


# *******************************************************************
# Environment and Config Variables. Change these variables as required.

my_fpath_courses = "../Data/main_coursera.csv"

my_fpath_skills_DataScientist = "../Data/Cosine-Distance/Single-Variable/CosDist_DataScientist.csv"

my_fpath_skills_SoftwareDevelopment = "../Data/Cosine-Distance/Single-Variable/CosDist_SoftwareDevelopment.csv" 

my_fpath_skills_DatabaseAdministrator = "../Data/Cosine-Distance/Single-Variable/CosDist_DatabaseAdministrator.csv"

my_fpath_skills_Cybersecurity = "../Data/Cosine-Distance/Single-Variable/CosDist_Cybersecurity.csv"

my_fpath_skills_FinancialAccountant = "../Data/Cosine-Distance/Single-Variable/CosDist_FinancialAccountant.csv"

my_fpath_skills_MachineLearning = "../Data/Cosine-Distance/Single-Variable/CosDist_MachineLearning.csv"

my_fpath_skills_Musician = "../Data/Cosine-Distance/Single-Variable/CosDist_Musician.csv"

my_fpath_skills_Dietitian = "../Data/Cosine-Distance/Single-Variable/CosDist_Dietitian.csv"

my_fpath_skills_Psychologist = "../Data/Cosine-Distance/Single-Variable/CosDist_Psychologist.csv"

# *******************************************************************


In [2]:
# Importing required modules/packages

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk, string


In [3]:
# Downloading the stopwords like i, me, and, is, the etc.

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/DV/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Loading courses and skills data from the CSV files

df_courses = pd.read_csv(my_fpath_courses)

df_DataScientist = pd.read_csv(my_fpath_skills_DataScientist)
df_DataScientist = df_DataScientist.drop('Role', 1)
df_DataScientist.columns = ['Course Id', 'DataScientist_Skill_Score', 'DataScientist_Role_Score', 'DataScientist_Keyword_Score']

df_SoftwareDevelopment = pd.read_csv(my_fpath_skills_SoftwareDevelopment)
df_SoftwareDevelopment = df_SoftwareDevelopment.drop('Role', 1)
df_SoftwareDevelopment.columns = ['Course Id','SoftwareDevelopment_Skill_Score', 'SoftwareDevelopment_Role_Score', 'SoftwareDevelopment_Keyword_Score']

df_DatabaseAdministrator = pd.read_csv(my_fpath_skills_DatabaseAdministrator)
df_DatabaseAdministrator = df_DatabaseAdministrator.drop('Role', 1)
df_DatabaseAdministrator.columns = ['Course Id','DatabaseAdministrator_Skill_Score', 'DatabaseAdministrator_Role_Score', 'DatabaseAdministrator_Keyword_Score']

df_Cybersecurity = pd.read_csv(my_fpath_skills_Cybersecurity)
df_Cybersecurity = df_Cybersecurity.drop('Role', 1)
df_Cybersecurity.columns = ['Course Id','Cybersecurity_Skill_Score', 'Cybersecurity_Role_Score', 'Cybersecurity_Keyword_Score']

df_FinancialAccountant = pd.read_csv(my_fpath_skills_FinancialAccountant)
df_FinancialAccountant = df_FinancialAccountant.drop('Role', 1)
df_FinancialAccountant.columns = ['Course Id','FinancialAccountant_Skill_Score', 'FinancialAccountant_Role_Score', 'FinancialAccountant_Keyword_Score']

df_MachineLearning = pd.read_csv(my_fpath_skills_MachineLearning)
df_MachineLearning = df_MachineLearning.drop('Role', 1)
df_MachineLearning.columns = ['Course Id','MachineLearning_Skill_Score', 'MachineLearning_Role_Score', 'MachineLearning_Keyword_Score']

df_Musician = pd.read_csv(my_fpath_skills_Musician)
df_Musician = df_Musician.drop('Role', 1)
df_Musician.columns = ['Course Id','Musician_Skill_Score', 'Musician_Role_Score', 'Musician_Keyword_Score']

df_Dietitian = pd.read_csv(my_fpath_skills_Dietitian)
df_Dietitian = df_Dietitian.drop('Role', 1)
df_Dietitian.columns = ['Course Id','Dietitian_Skill_Score', 'Dietitian_Role_Score','Dietitian_Keyword_Score']

df_Psychologist = pd.read_csv(my_fpath_skills_Psychologist)
df_Psychologist = df_Psychologist.drop('Role', 1)
df_Psychologist.columns = ['Course Id','Psychologist_Skill_Score', 'Psychologist_Role_Score', 'Psychologist_Keyword_Score']

In [5]:
# Merging the csv files

df_cosdist = df_DataScientist.merge(df_SoftwareDevelopment, on = 'Course Id', how = 'outer')

df_cosdist = df_cosdist.merge(df_DatabaseAdministrator, on = 'Course Id', how = 'outer')

df_cosdist = df_cosdist.merge(df_Cybersecurity, on = 'Course Id', how = 'outer')

df_cosdist = df_cosdist.merge(df_FinancialAccountant, on = 'Course Id', how = 'outer')

df_cosdist = df_cosdist.merge(df_MachineLearning, on = 'Course Id', how = 'outer')

df_cosdist = df_cosdist.merge(df_Musician, on = 'Course Id', how = 'outer')

df_cosdist = df_cosdist.merge(df_Dietitian, on = 'Course Id', how = 'outer')

df_cosdist = df_cosdist.merge(df_Psychologist, on = 'Course Id', how = 'outer')


In [6]:
# Exploring data dimensionality, feature names, and feature types.

print(df_courses.shape,"\n")

print(df_cosdist.shape,"\n")

print(df_courses.columns, "\n")

print(df_cosdist.shape,"\n")

print(df_courses.describe(), "\n")

print(df_cosdist.describe(), "\n")


(2213, 19) 

(2213, 28) 

Index(['Unnamed: 0', 'Course Id', 'Course Name', 'Course Description', 'Slug',
       'Provider', 'Universities/Institutions', 'Parent Subject',
       'Child Subject', 'Category', 'Url', 'Length', 'Language',
       'Credential Name', 'Rating', 'Number of Ratings', 'Certificate',
       'Workload', 'Course Keywords'],
      dtype='object') 

(2213, 28) 

        Unnamed: 0    Course Id      Length       Rating  Number of Ratings  \
count  2213.000000  2213.000000  964.000000  2213.000000        2213.000000   
mean   1106.000000  4816.998192    6.063278     2.352785          10.321735   
std     638.982394  3033.878865    2.724669     2.129134         110.680382   
min       0.000000   303.000000    1.000000     0.000000           0.000000   
25%     553.000000  1829.000000    4.000000     0.000000           0.000000   
50%    1106.000000  4880.000000    6.000000     3.000000           1.000000   
75%    1659.000000  7329.000000    7.000000     4.428571       

In [7]:
# Quick check to see if the dataframe showing the right results

df_cosdist.head(20)

Course Id  DataScientist_Skill_Score  DataScientist_Role_Score  \
0         303                   0.307459                  0.744684   
1         305                   0.232071                  0.535167   
2         306                   0.323008                  0.449385   
3         307                   0.309164                  0.416260   
4         308                   0.300722                  0.284999   
5         309                   0.307471                  0.522170   
6         316                   0.334175                  0.310164   
7         317                   0.272255                  0.375398   
8         318                   0.281119                  0.473992   
9         322                   0.310068                  0.642344   
10        328                   0.291573                  0.204527   
11        329                   0.292197                  0.426543   
12        331                   0.323042                  0.260175   
13        332                   0.339114                  0.560099   
14        333                   0.334162                  0.560099   
15        334                   0.356406                  0.890793   
16        335                   0.285643                  0.656167   
17        336                   0.299945                  0.377455   
18        337                   0.304868                  0.501611   
19        338                   0.302569                  0.520878   

    DataScientist_Keyword_Score  SoftwareDevelopment_Skill_Score  \
0                      0.544460                         0.237837   
1                      0.460375                         0.255782   
2                      0.219802                         0.227007   
3                      0.648603                         0.228633   
4                      0.602987                         0.239965   
5                      0.757452                         0.248328   
6                      0.355386                         0.248898   
7                      0.364093                         0.236654   
8                      0.571874                         0.233110   
9                      0.753330                         0.241869   
10                     0.540851                         0.246556   
11                     0.343058                         0.220996   
12                     0.379706                         0.237752   
13                     0.506005                         0.247362   
14                     0.428784                         0.246957   
15                     0.439425                         0.234769   
16                     0.684342                         0.208457   
17                     0.219802                         0.234933   
18                     0.559390                         0.228512   
19                     0.723361                         0.228785   

    SoftwareDevelopment_Role_Score  SoftwareDevelopment_Keyword_Score  \
0                         0.603931                           0.518541   
1                         0.693831                           0.613591   
2                         0.576043                           0.074077   
3                         0.492073                           0.519089   
4                         0.593450                           0.530510   
5                         0.724140                           0.667862   
6                         0.610637                           0.343334   
7                         0.532408                           0.405278   
8                         0.530605                           0.539747   
9                         0.780723                           0.766033   
10                        0.548851                           0.615308   
11                        0.404026                           0.352089   
12                        0.386239                           0.332682   
13                        0.381027                           0.404344   
14       

In [8]:
# Joining two dataframes - Courses and the Cosein Similarity Results based on the 'Course Id' variable. 
# Inner joins: Joins two tables with the common rows. This is a set operateion.

df_courses_score = df_courses.merge(df_cosdist, on ='Course Id', how='inner')

print(df_courses_score.shape,"\n")

(2213, 46) 



In [9]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: DATA SCIENTIST

my_DataScientist_skill_score = 'DataScientist_Skill_Score'
my_DataScientist_final_score = 'DataScientist_Final_Score'
y_actu_DataScientist         = ''
y_pred_DataScientist         = ''

df_courses_score[[my_DataScientist_final_score]] = df_courses_score[[my_DataScientist_skill_score]]

df_courses_score['DataScientist_Predict'] = (df_courses_score[[my_DataScientist_final_score]] >= 0.5)

df_courses_score['DataScientist_Label'] = df_courses_score.Category.isin(label_DataScientist)

y_pred_DataScientist = pd.Series(df_courses_score['DataScientist_Predict'], name='Predicted')

y_actu_DataScientist = pd.Series(df_courses_score['DataScientist_Label'], name='Actual')

df_confusion_DataScientist = pd.crosstab(y_actu_DataScientist, y_pred_DataScientist , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [10]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: SOFTWARE ENGINEER/DEVELOPER

my_SoftwareDevelopment_skill_score = 'SoftwareDevelopment_Skill_Score'
my_SoftwareDevelopment_final_score = 'SoftwareDevelopment_Final_Score'
y_actu_SoftwareDevelopment         = ''
y_pred_SoftwareDevelopment         = ''

df_courses_score[[my_SoftwareDevelopment_final_score]] = df_courses_score[[my_SoftwareDevelopment_skill_score]]

df_courses_score['SoftwareDevelopment_Predict'] = (df_courses_score[[my_SoftwareDevelopment_final_score]] >= 0.5)

df_courses_score['SoftwareDevelopment_Label'] = df_courses_score.Category.isin(label_SoftwareDevelopment)

y_pred_SoftwareDevelopment = pd.Series(df_courses_score['SoftwareDevelopment_Predict'], name='Predicted')

y_actu_SoftwareDevelopment = pd.Series(df_courses_score['SoftwareDevelopment_Label'], name='Actual')

df_confusion_SoftwareDevelopment = pd.crosstab(y_actu_SoftwareDevelopment, y_pred_SoftwareDevelopment , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [11]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: DATABASE DEVELOPER/ADMINISTRATOR

my_DatabaseAdministrator_skill_score = 'DatabaseAdministrator_Skill_Score'
my_DatabaseAdministrator_final_score = 'DatabaseAdministrator_Final_Score'
y_actu_DatabaseAdministrator         = ''
y_pred_DatabaseAdministrator         = ''

df_courses_score[[my_DatabaseAdministrator_final_score]] = df_courses_score[[my_DatabaseAdministrator_skill_score]]

df_courses_score['DatabaseAdministrator_Predict'] = (df_courses_score[[my_DatabaseAdministrator_final_score]] >= 0.5)

df_courses_score['DatabaseAdministrator_Label'] = df_courses_score.Category.isin(label_DatabaseAdministrator)

y_pred_DatabaseAdministrator = pd.Series(df_courses_score['DatabaseAdministrator_Predict'], name='Predicted')

y_actu_DatabaseAdministrator = pd.Series(df_courses_score['DatabaseAdministrator_Label'], name='Actual')

df_confusion_DatabaseAdministrator = pd.crosstab(y_actu_DatabaseAdministrator, y_pred_DatabaseAdministrator , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [12]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: CYBERSECURITY CONSULTANT

my_Cybersecurity_skill_score = 'Cybersecurity_Skill_Score'
my_Cybersecurity_final_score = 'Cybersecurity_Final_Score'
y_actu_Cybersecurity         = ''
y_pred_Cybersecurity         = ''

df_courses_score[[my_Cybersecurity_final_score]] = df_courses_score[[my_Cybersecurity_skill_score]]

df_courses_score['Cybersecurity_Predict'] = (df_courses_score[[my_Cybersecurity_final_score]] >= 0.5)

df_courses_score['Cybersecurity_Label'] = df_courses_score.Category.isin(label_Cybersecurity)

y_pred_Cybersecurity = pd.Series(df_courses_score['Cybersecurity_Predict'], name='Predicted')

y_actu_Cybersecurity = pd.Series(df_courses_score['Cybersecurity_Label'], name='Actual')

df_confusion_Cybersecurity = pd.crosstab(y_actu_Cybersecurity, y_pred_Cybersecurity , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [13]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: FINANCIAL ACCOUNTANT

my_FinancialAccountant_skill_score = 'FinancialAccountant_Skill_Score'
my_FinancialAccountant_final_score = 'FinancialAccountant_Final_Score'
y_actu_FinancialAccountant         = ''
y_pred_FinancialAccountant         = ''

df_courses_score[[my_FinancialAccountant_final_score]] = df_courses_score[[my_FinancialAccountant_skill_score]]

df_courses_score['FinancialAccountant_Predict'] = (df_courses_score[[my_FinancialAccountant_final_score]] >= 0.5)

df_courses_score['FinancialAccountant_Label'] = df_courses_score.Category.isin(label_FinancialAccountant)

y_pred_FinancialAccountant = pd.Series(df_courses_score['FinancialAccountant_Predict'], name='Predicted')

y_actu_FinancialAccountant = pd.Series(df_courses_score['FinancialAccountant_Label'], name='Actual')

df_confusion_FinancialAccountant = pd.crosstab(y_actu_FinancialAccountant, y_pred_FinancialAccountant , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [14]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: MACHINE LEARNING ENGINEER

my_MachineLearning_skill_score = 'MachineLearning_Skill_Score'
my_MachineLearning_final_score = 'MachineLearning_Final_Score'
y_actu_MachineLearning         = ''
y_pred_MachineLearning         = ''

df_courses_score[[my_MachineLearning_final_score]] = df_courses_score[[my_MachineLearning_skill_score]]

df_courses_score['MachineLearning_Predict'] = (df_courses_score[[my_MachineLearning_final_score]] >= 0.5)

df_courses_score['MachineLearning_Label'] = df_courses_score.Category.isin(label_MachineLearning)

y_pred_MachineLearning = pd.Series(df_courses_score['MachineLearning_Predict'], name='Predicted')

y_actu_MachineLearning = pd.Series(df_courses_score['MachineLearning_Label'], name='Actual')

df_confusion_MachineLearning = pd.crosstab(y_actu_MachineLearning, y_pred_MachineLearning , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [15]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: MUSICIAN

my_Musician_skill_score = 'Musician_Skill_Score'
my_Musician_final_score = 'Musician_Final_Score'
y_actu_Musician         = ''
y_pred_Musician         = ''

df_courses_score[[my_Musician_final_score]] = df_courses_score[[my_Musician_skill_score]]

df_courses_score['Musician_Predict'] = (df_courses_score[[my_Musician_final_score]] >= 0.5)

df_courses_score['Musician_Label'] = df_courses_score.Category.isin(label_Musician)

y_pred_Musician = pd.Series(df_courses_score['Musician_Predict'], name='Predicted')

y_actu_Musician = pd.Series(df_courses_score['Musician_Label'], name='Actual')

df_confusion_Musician = pd.crosstab(y_actu_Musician, y_pred_Musician , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [16]:
# Tranforming and shaping the data to create the confusion matrix for the ROLE: NUTRITIONIST/DIETITIAN

my_Dietitian_skill_score = 'Dietitian_Skill_Score'
my_Dietitian_final_score = 'Dietitian_Final_Score'
y_actu_Dietitian         = ''
y_pred_Dietitian         = ''

df_courses_score[[my_Dietitian_final_score]] = df_courses_score[[my_Dietitian_skill_score]]

df_courses_score['Dietitian_Predict'] = (df_courses_score[[my_Dietitian_final_score]] >= 0.5)

df_courses_score['Dietitian_Label'] = df_courses_score.Category.isin(label_Dietitian)

y_pred_Dietitian = pd.Series(df_courses_score['Dietitian_Predict'], name='Predicted')

y_actu_Dietitian = pd.Series(df_courses_score['Dietitian_Label'], name='Actual')

df_confusion_Dietitian = pd.crosstab(y_actu_Dietitian, y_pred_Dietitian , rownames=['Actual'], colnames=['Predicted'], margins=False)


In [17]:
df_confusion_DataScientist


Predicted  False
Actual          
False       2131
True          82

In [18]:
df_confusion_SoftwareDevelopment

Predicted  False
Actual          
False       2092
True         121

In [19]:
df_confusion_DatabaseAdministrator

Predicted  False  True 
Actual                 
False       2196      6
True          11      0

In [20]:
df_confusion_Cybersecurity

Predicted  False  True 
Actual                 
False        156   2027
True           0     30

In [21]:
df_confusion_FinancialAccountant

Predicted  False  True 
Actual                 
False        105   2005
True           2    101

In [22]:
df_confusion_MachineLearning

Predicted  False
Actual          
False       2189
True          24

In [23]:
df_confusion_Musician

Predicted  False  True 
Actual                 
False       2174      2
True          37      0

In [24]:
df_confusion_Dietitian

Predicted  False  True 
Actual                 
False        966   1221
True           5     21

In [25]:
# Performance summary for the ROLE: DATA SCIENTIST


try:
    tn_DataScientist = df_confusion_DataScientist.iloc[0][False]
except:
    tn_DataScientist = 0
    
try:
    tp_DataScientist =  df_confusion_DataScientist.iloc[1][True]
except:
    tp_DataScientist = 0

    
try:
    fn_DataScientist = df_confusion_DataScientist.iloc[1][False]
except:
    fn_DataScientist = 0
    
try:
    fp_DataScientist =  df_confusion_DataScientist.iloc[0][True]
except:
    fp_DataScientist = 0  
    
    
total_count_DataScientist = tn_DataScientist + tp_DataScientist + fn_DataScientist + fp_DataScientist

print('Data Scientist Accuracy Rate : ', '{0:.2f}'.format((tn_DataScientist + tp_DataScientist) / total_count_DataScientist * 100))

print('Data Scientist Misclassifcation Rate : ',  '{0:.2f}'.format((fn_DataScientist + fp_DataScientist) / total_count_DataScientist * 100))

print('Data Scientist True Positive Rate : ',  '{0:.2f}'.format(tp_DataScientist / (tp_DataScientist + fn_DataScientist) * 100))

print('Data Scientist False Positive Rate : ',  '{0:.2f}'.format(fp_DataScientist / (tn_DataScientist + fp_DataScientist) * 100))


Data Scientist Accuracy Rate :  96.29
Data Scientist Misclassifcation Rate :  3.71
Data Scientist True Positive Rate :  0.00
Data Scientist False Positive Rate :  0.00


In [26]:
# Performance summary for the ROLE: DATA SCIENTIST


try:
    tn_SoftwareDevelopment = df_confusion_SoftwareDevelopment.iloc[0][False]
except:
    tn_SoftwareDevelopment = 0
    
try:
    tp_SoftwareDevelopment =  df_confusion_SoftwareDevelopment.iloc[1][True]
except:
    tp_SoftwareDevelopment = 0

    
try:
    fn_SoftwareDevelopment = df_confusion_SoftwareDevelopment.iloc[1][False]
except:
    fn_SoftwareDevelopment = 0
    
try:
    fp_SoftwareDevelopment =  df_confusion_SoftwareDevelopment.iloc[0][True]
except:
    fp_SoftwareDevelopment = 0  
    
    
total_count_SoftwareDevelopment = tn_SoftwareDevelopment + tp_SoftwareDevelopment + fn_SoftwareDevelopment + fp_SoftwareDevelopment

print('Software Engineer Accuracy Rate : ', '{0:.2f}'.format((tn_SoftwareDevelopment + tp_SoftwareDevelopment) / total_count_SoftwareDevelopment * 100))

print('Software Engineer Misclassifcation Rate : ',  '{0:.2f}'.format((fn_SoftwareDevelopment + fp_SoftwareDevelopment) / total_count_SoftwareDevelopment * 100))

print('Software Engineer True Positive Rate : ',  '{0:.2f}'.format(tp_SoftwareDevelopment / (tp_SoftwareDevelopment + fn_SoftwareDevelopment) * 100))

print('Software Engineer False Positive Rate : ',  '{0:.2f}'.format(fp_SoftwareDevelopment / (tn_SoftwareDevelopment + fp_SoftwareDevelopment) * 100))


Software Engineer Accuracy Rate :  94.53
Software Engineer Misclassifcation Rate :  5.47
Software Engineer True Positive Rate :  0.00
Software Engineer False Positive Rate :  0.00


In [27]:
# Performance summary for the ROLE: DATABASE DEVELOPER/ ADMINISTRATOR


try:
    tn_DatabaseAdministrator = df_confusion_DatabaseAdministrator.iloc[0][False]
except:
    tn_DatabaseAdministrator = 0
    
try:
    tp_DatabaseAdministrator =  df_confusion_DatabaseAdministrator.iloc[1][True]
except:
    tp_DatabaseAdministrator = 0

    
try:
    fn_DatabaseAdministrator = df_confusion_DatabaseAdministrator.iloc[1][False]
except:
    fn_DatabaseAdministrator = 0
    
try:
    fp_DatabaseAdministrator =  df_confusion_DatabaseAdministrator.iloc[0][True]
except:
    fp_DatabaseAdministrator = 0  
    
    
total_count_DatabaseAdministrator = tn_DatabaseAdministrator + tp_DatabaseAdministrator + fn_DatabaseAdministrator + fp_DatabaseAdministrator

print('Database Administrator Accuracy Rate : ', '{0:.2f}'.format((tn_DatabaseAdministrator + tp_DatabaseAdministrator) / total_count_DatabaseAdministrator * 100))

print('Database Administrator Misclassifcation Rate : ',  '{0:.2f}'.format((fn_DatabaseAdministrator + fp_DatabaseAdministrator) / total_count_DatabaseAdministrator * 100))

print('Database Administrator True Positive Rate : ',  '{0:.2f}'.format(tp_DatabaseAdministrator / (tp_DatabaseAdministrator + fn_DatabaseAdministrator) * 100))

print('Database Administrator False Positive Rate : ',  '{0:.2f}'.format(fp_DatabaseAdministrator / (tn_DatabaseAdministrator + fp_DatabaseAdministrator) * 100))


Database Administrator Accuracy Rate :  99.23
Database Administrator Misclassifcation Rate :  0.77
Database Administrator True Positive Rate :  0.00
Database Administrator False Positive Rate :  0.27


In [28]:
# Performance summary for the ROLE: CYBERSECURITY CONSULTANT


try:
    tn_Cybersecurity = df_confusion_Cybersecurity.iloc[0][False]
except:
    tn_Cybersecurity = 0
    
try:
    tp_Cybersecurity =  df_confusion_Cybersecurity.iloc[1][True]
except:
    tp_Cybersecurity = 0

    
try:
    fn_Cybersecurity = df_confusion_Cybersecurity.iloc[1][False]
except:
    fn_Cybersecurity = 0
    
try:
    fp_Cybersecurity =  df_confusion_Cybersecurity.iloc[0][True]
except:
    fp_Cybersecurity = 0  
    
    
total_count_Cybersecurity = tn_Cybersecurity + tp_Cybersecurity + fn_Cybersecurity + fp_Cybersecurity

print('Cybersecurity Consultant Accuracy Rate : ', '{0:.2f}'.format((tn_Cybersecurity + tp_Cybersecurity) / total_count_Cybersecurity * 100))

print('Cybersecurity Consultant Misclassifcation Rate : ',  '{0:.2f}'.format((fn_Cybersecurity + fp_Cybersecurity) / total_count_Cybersecurity * 100))

print('Cybersecurity Consultant True Positive Rate : ',  '{0:.2f}'.format(tp_Cybersecurity / (tp_Cybersecurity + fn_Cybersecurity) * 100))

print('Cybersecurity Consultant False Positive Rate : ',  '{0:.2f}'.format(fp_Cybersecurity / (tn_Cybersecurity + fp_Cybersecurity) * 100))


Cybersecurity Consultant Accuracy Rate :  8.40
Cybersecurity Consultant Misclassifcation Rate :  91.60
Cybersecurity Consultant True Positive Rate :  100.00
Cybersecurity Consultant False Positive Rate :  92.85


In [29]:
# Performance summary for the ROLE: FINANCIAL ACCOUNTANT


try:
    tn_FinancialAccountant = df_confusion_FinancialAccountant.iloc[0][False]
except:
    tn_FinancialAccountant = 0
    
try:
    tp_FinancialAccountant =  df_confusion_FinancialAccountant.iloc[1][True]
except:
    tp_FinancialAccountant = 0

    
try:
    fn_FinancialAccountant = df_confusion_FinancialAccountant.iloc[1][False]
except:
    fn_FinancialAccountant = 0
    
try:
    fp_FinancialAccountant =  df_confusion_FinancialAccountant.iloc[0][True]
except:
    fp_FinancialAccountant = 0  
    
    
total_count_FinancialAccountant = tn_FinancialAccountant + tp_FinancialAccountant + fn_FinancialAccountant + fp_FinancialAccountant

print('Financial Accountant Consultant Accuracy Rate : ', '{0:.2f}'.format((tn_FinancialAccountant + tp_FinancialAccountant) / total_count_FinancialAccountant * 100))

print('Financial Accountant Consultant Misclassifcation Rate : ',  '{0:.2f}'.format((fn_FinancialAccountant + fp_FinancialAccountant) / total_count_FinancialAccountant * 100))

print('Financial Accountant Consultant True Positive Rate : ',  '{0:.2f}'.format(tp_FinancialAccountant / (tp_FinancialAccountant + fn_FinancialAccountant) * 100))

print('Financial Accountant Consultant False Positive Rate : ',  '{0:.2f}'.format(fp_FinancialAccountant / (tn_FinancialAccountant + fp_FinancialAccountant) * 100))


Financial Accountant Consultant Accuracy Rate :  9.31
Financial Accountant Consultant Misclassifcation Rate :  90.69
Financial Accountant Consultant True Positive Rate :  98.06
Financial Accountant Consultant False Positive Rate :  95.02


In [30]:
# Performance summary for the ROLE: MACHINE LEARNING ENGINEER


try:
    tn_MachineLearning = df_confusion_MachineLearning.iloc[0][False]
except:
    tn_MachineLearning = 0
    
try:
    tp_MachineLearning =  df_confusion_MachineLearning.iloc[1][True]
except:
    tp_MachineLearning = 0

    
try:
    fn_MachineLearning = df_confusion_MachineLearning.iloc[1][False]
except:
    fn_MachineLearning = 0
    
try:
    fp_MachineLearning =  df_confusion_MachineLearning.iloc[0][True]
except:
    fp_MachineLearning = 0  
    
    
total_count_MachineLearning = tn_MachineLearning + tp_MachineLearning + fn_MachineLearning + fp_MachineLearning

print('Machine Learning Engineer Accuracy Rate : ', '{0:.2f}'.format((tn_MachineLearning + tp_MachineLearning) / total_count_MachineLearning * 100))

print('Machine Learning Engineer Misclassifcation Rate : ',  '{0:.2f}'.format((fn_MachineLearning + fp_MachineLearning) / total_count_MachineLearning * 100))

print('Machine Learning Engineer True Positive Rate : ',  '{0:.2f}'.format(tp_MachineLearning / (tp_MachineLearning + fn_MachineLearning) * 100))

print('Machine Learning Engineer False Positive Rate : ',  '{0:.2f}'.format(fp_MachineLearning / (tn_MachineLearning + fp_MachineLearning) * 100))


Machine Learning Engineer Accuracy Rate :  98.92
Machine Learning Engineer Misclassifcation Rate :  1.08
Machine Learning Engineer True Positive Rate :  0.00
Machine Learning Engineer False Positive Rate :  0.00


In [31]:
# Performance summary for the ROLE: MUSICIAN


try:
    tn_Musician = df_confusion_Musician.iloc[0][False]
except:
    tn_Musician = 0
    
try:
    tp_Musician =  df_confusion_Musician.iloc[1][True]
except:
    tp_Musician = 0

    
try:
    fn_Musician = df_confusion_Musician.iloc[1][False]
except:
    fn_Musician = 0
    
try:
    fp_Musician =  df_confusion_Musician.iloc[0][True]
except:
    fp_Musician = 0  
    
    
total_count_Musician = tn_Musician + tp_Musician + fn_Musician + fp_Musician

print('Musician Accuracy Rate : ', '{0:.2f}'.format((tn_Musician + tp_Musician) / total_count_Musician * 100))

print('Musician Misclassifcation Rate : ',  '{0:.2f}'.format((fn_Musician + fp_Musician) / total_count_Musician * 100))

print('Musician True Positive Rate : ',  '{0:.2f}'.format(tp_Musician / (tp_Musician + fn_Musician) * 100))

print('Musician False Positive Rate : ',  '{0:.2f}'.format(fp_Musician / (tn_Musician + fp_Musician) * 100))


Musician Accuracy Rate :  98.24
Musician Misclassifcation Rate :  1.76
Musician True Positive Rate :  0.00
Musician False Positive Rate :  0.09


In [32]:
# Performance summary for the ROLE: DIETITIAN


try:
    tn_Dietitian = df_confusion_Dietitian.iloc[0][False]
except:
    tn_Dietitian = 0
    
try:
    tp_Dietitian =  df_confusion_Dietitian.iloc[1][True]
except:
    tp_Dietitian = 0

    
try:
    fn_Dietitian = df_confusion_Dietitian.iloc[1][False]
except:
    fn_Dietitian = 0
    
try:
    fp_Dietitian =  df_confusion_Dietitian.iloc[0][True]
except:
    fp_Dietitian = 0  
    
    
total_count_Dietitian = tn_Dietitian + tp_Dietitian + fn_Dietitian + fp_Dietitian

print('Dietitian Accuracy Rate : ', '{0:.2f}'.format((tn_Dietitian + tp_Dietitian) / total_count_Dietitian * 100))

print('Dietitian Misclassifcation Rate : ',  '{0:.2f}'.format((fn_Dietitian + fp_Dietitian) / total_count_Dietitian * 100))

print('Dietitian True Positive Rate : ',  '{0:.2f}'.format(tp_Dietitian / (tp_Dietitian + fn_Dietitian) * 100))

print('Dietitian False Positive Rate : ',  '{0:.2f}'.format(fp_Dietitian / (tn_Dietitian + fp_Dietitian) * 100))


Dietitian Accuracy Rate :  44.60
Dietitian Misclassifcation Rate :  55.40
Dietitian True Positive Rate :  80.77
Dietitian False Positive Rate :  55.83


### End of the Notebook. Thank you!